In [2]:
# %pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [19]:
import tensorflow as tf
import tensorflow_text as tf_text
import pathlib
import pandas as pd
import os
import io
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [3]:
url = 'https://go.aws/314bBDq'
df = pd.read_csv(url)
df.describe()

,stars,hour_of_day,review_len,review_nb_words
count,299635.000000,299635.000000,299635.000000,299635.000000
mean,4.573287,14.860373,10.579902,1.881930
std,0.907353,6.116968,73.734467,13.315102
min,1.000000,0.000000,0.000000,0.000000
25%,5.000000,11.000000,0.000000,0.000000
50%,5.000000,16.000000,0.000000,0.000000
75%,5.000000,20.000000,0.000000,0.000000
max,5.000000,23.000000,2915.000000,571.000000


In [4]:
df.head(10)

,user_id,review,stars,date_format,time_of_day,hour_of_day,day_of_week,review_format,review_lang,month_year,review_len,review_nb_words
0,efb62a167fee5cf3678b24427de8e31f,"Génial, fabuleux, exceptionnel ! J'aimerais qu...",5,2017-09-29 18:17:00,18:17,18,Ven,génial fabuleux exceptionnel j aimerais qu...,french,2017-09,115,19
1,e3be4f9c9e0b9572bfb2a5f88497bb14,NaN,2,2017-09-29 17:29:00,17:29,17,Ven,NaN,NaN,2017-09,0,0
2,1b8e5760162d867e9b9ca80f645bdc60,"Toujours aussi magic, féerique !",5,2017-09-29 16:46:00,16:46,16,Ven,toujours aussi magic féerique,french,2017-09,32,4
3,fa330e5891a1bb486c3e9bf95c098726,NaN,5,2017-09-29 15:52:00,15:52,15,Ven,NaN,NaN,2017-09,0,0
4,c1a693206aee1a2412d4bd9e45b80ec5,NaN,3,2017-09-29 15:29:00,15:29,15,Ven,NaN,NaN,2017-09,0,0
5,b48f8f41f4f923b3b96362d9bd729a82,NaN,5,2017-09-29 15:09:00,15:09,15,Ven,NaN,NaN,2017-09,0,0
6,e3fa8d2f3bf2934cfc136621efe498ab,NaN,5,2017-09-29 15:07:00,15:07,15,Ven,NaN,NaN,2017-09,0,0
7,42116b0a64a4995a56568da77bed067b,NaN,4,2017-09-29 12:32:00,12:32,12,Ven,NaN,NaN,2017-09,0,0
8,9267478dabfcbc1f6e67459660ef04d5,NaN,5,2017-09-29 10:56:00,10:56,10,Ven,NaN,NaN,2017-09,0,0
9,ff017733b38aad87068b425f45962a19,NaN,5,2017-09-29 10:22:00,10:22,10,Ven,NaN,NaN,2017-09,0,0


In [5]:
df.columns

Index(['user_id', 'review', 'stars', 'date_format', 'time_of_day',
       'hour_of_day', 'day_of_week', 'review_format', 'review_lang',
       'month_year', 'review_len', 'review_nb_words'],
      dtype='object')

In [6]:
df['review_lang'].unique()

array(['french', nan, 'romanian', 'estonian', 'slovenian-iso8859_2',
       'english', 'dutch', 'czech-iso8859_2', 'danish',
       'slovak-windows1250', 'frisian', 'scots', 'portuguese', 'latin',
       'lithuanian', 'rumantsch', 'polish', 'russian-koi8_r',
       'middle_frisian', 'norwegian', 'swedish', 'spanish', 'catalan',
       'esperanto', 'italian', 'russian-windows1251', 'russian-iso8859_5',
       'tagalog', 'greek-iso8859-7', 'turkish', 'breton', 'german',
       'hebrew-iso8859_8', 'bosnian', 'hungarian', 'afrikaans',
       'sanskrit', 'irish', 'albanian', 'indonesian', 'latvian', 'manx',
       'finnish', 'belarus-windows1251', 'welsh', 'slovenian-ascii',
       'slovak-ascii', 'swahili', 'nepali', 'bulgarian-iso8859_5',
       'croatian-ascii', 'icelandic', 'serbian-ascii', 'ukrainian-koi8_r',
       'malay', 'scots_gaelic', 'basque'], dtype=object)

In [7]:
df = df.loc[df['review_lang'] == 'french'].loc[:, ['review', 'stars']]
df.head()

,review,stars
0,"Génial, fabuleux, exceptionnel ! J'aimerais qu...",5
2,"Toujours aussi magic, féerique !",5
11,En vacances en région parisienne nous nous som...,2
12,Tropbeaufinalpleinlesyeuxoreil,5
23,L'univers Disney reste merveilleux. Toutefois ...,4


In [8]:
df.describe()

,stars
count,8474.000000
mean,4.144324
std,1.223539
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [9]:
len(df)

8474

In [10]:
# !python -m spacy download fr_core_news_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 16.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [12]:
# nlp.Defaults.stop_words

In [13]:
doc = nlp("je la laisse, je veux la laisser, je l'ai laissée, je me suis laissé aller")
for token in doc:
    print(token.text, token.pos_, token.dep_)

je PRON nsubj
la PRON obj
laisse VERB advcl
, PUNCT punct
je PRON nsubj
veux VERB advcl
la PRON obj
laisser VERB xcomp
, PUNCT punct
je PRON nsubj
l' PRON obj
ai AUX aux:tense
laissée VERB conj
, PUNCT punct
je PRON nsubj:pass
me PRON expl:comp
suis AUX aux:pass
laissé VERB ROOT
aller VERB xcomp


In [14]:
[token.lemma_ for token in doc if not token.is_stop]

['laisse', ',', 'vouloir', ',', 'laisser', ',', 'laisser', 'aller']

Back to the data

In [15]:
rev = df['review'].apply(lambda s: ''.join(c for c in s if (c.isalnum() or c in " ',")))
rev = rev.apply(str.strip).apply(str.casefold).apply(lambda s: s.replace('  ', ' '))

In [16]:
rev.head(10)

,review
0,"génial, fabuleux, exceptionnel j'aimerais que ..."
2,"toujours aussi magic, féerique"
11,en vacances en région parisienne nous nous som...
12,tropbeaufinalpleinlesyeuxoreil
23,l'univers disney reste merveilleux toutefois j...
29,nous avons passé 3 jours dans les 2 parcs et 2...
37,disney c'est le rève et le dépaysement assuré ...
40,"mal organisé, trop d'attente pour certaines at..."
42,un séjour magique pour ma princesse de 5 ans m...
44,je comprend pas très bien les zones fumeurs je...


In [27]:
def lemmatize(s: str)-> str:
  # lemmatized = ' '.join(token.lemma_ for token in nlp(s) if not token.is_stop)
  return ' '.join(token.lemma_ for token in nlp(s) if not token.is_stop)

clean_rev = rev.apply(lemmatize)

In [23]:
clean_rev.apply(bool).describe()

,review
count,8474
unique,2
top,True
freq,8473


In [26]:
clean_rev

,review
0,"génial , fabuleux , exceptionnel je aimer que ..."
2,"toujours aussi magic , féerique"
11,en vacance en région parisien nous nous être d...
12,tropbeaufinalpleinlesyeuxoreil
23,le univers disney reste merveilleux toutefois ...
...,...
295057,toujours aussi magique même si à le fin de séj...
295549,séjour au top mon enfant le plus heureux avoir...
298475,magnifique un monde parfait span class47e3 5mf...
298832,oui je avoir aimer car je adorer disney et tou...


In [62]:
clean_rev = clean_rev.loc[clean_rev.apply(bool)]

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, char_level=False, oov_token='out_of_vocab')


In [63]:
clean_rev.to_numpy()

array(['génial , fabuleux , exceptionnel aimer walt disney réveiller admirer laisser',
       'magic , féerique',
       'vacance région parisien être décider visiter parc reconnaître mondialement br beal décor , parc propre , temps attente acceptable',
       ...,
       'magnifique monde parfait span class47e3 5mfr titleémoticône heartimg alt classimg height24 rolepresentation srchttpsstaticxxfbcdnnetimagesemojiphpv9f702242764png width24span ariahiddentrue class7oelt3spanspan',
       'oui aimer adorer disney toucher univers grand enfant br premier séjour moibr séquoia lodg   top',
       'disney minimum 1 fois saison magique printemps hiver , petit faible noël vraiment magique , retomber enfance perso , jamais être décevoir , faire attente long attraction casser nez heure file attraction tombe panne arriver but désagrément vite effacer restebr repartir envie revenir bien vite'],
      dtype=object)

In [64]:
tokenizer.fit_on_texts(clean_rev.to_numpy())
token_rev = tokenizer.texts_to_sequences(clean_rev.to_numpy())

In [49]:
token_rev[0]

[111, 606, 499, 106, 500, 4, 676]

In [54]:
import numpy as np
np.std([len(t) for t in token_rev])

19.998884676776267

In [61]:
import tensorflow.keras as keras
from keras.utils import pad_sequences

data = pad_sequences(token_rev, maxlen=50, padding='post', truncating='post')
data

TypeError: pad_sequences() got an unexpected keyword argument 'oov_token'